In [1]:
import nltk, pickle, string
from nltk.corpus import stopwords
import re
import os
from tika import parser
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


from textblob.classifiers import NaiveBayesClassifier as NBC
from textblob import TextBlob


Using TensorFlow backend.


In [2]:
def clean_doc(doc):
    """
    Cleaning a document by several methods:
        - Lowercase
        - Removing whitespaces
        - Removing stopwords
        - Removing punctuations
        
    """
    stop_words = set(stopwords.words('english'))
    
    # Lowercase
    doc = doc.lower()
   
    tokens = doc.translate({ord(c):"" for c in "\u200b\uf020\u2028\xa0\uf0e0\uf095\uf041\uf0e1\uf0b7\xad"})
    tokens = tokens.translate({ord(c):" " for c in "[):,·](;•●■♦▪"})
    tokens = tokens.translate({ord(c):"f" for c in "�"})
    
    # Removing multiple whitespaces
    tokens = re.sub(r"\?", " \? ", tokens)
    
    # Split in tokens
    tokens = tokens.split()
    
    # Remove Stopwords
    tokens = [w for w in tokens if not w in stop_words]
    # Remove punctuation
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    
    return ' '.join(tokens)

In [3]:
def LoadData_nontech():
    label_nontech=[]
    i=0
    for foldername in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/nontech"):
        for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/nontech/"+foldername):
    
            try:
                print (i, foldername, file)
                parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/nontech/"+foldername+"/"+file)

                resume_contents=clean_doc(parsedPDF['content'])
            #Data = resume_contents.encode('utf-8')    
                label_nontech.append((resume_contents,foldername))
            except UnicodeEncodeError:
                print ('Unicode error:', file)
            i=i+1
    #print (label_resume)
    return(label_nontech)

In [4]:
label_nontech=LoadData_nontech()

0 marketing Bhaskar_Singh_Kolkata_15.00_yrs.doc
1 marketing Samuel_Raja_Velapula_Bengaluru___Bangalore_5.01_yrs.docx
2 marketing Pranshu_Gupta_Delhi_10.00_yrs.doc
3 marketing Rachita_Vig_Pune_0.00_yrs.docx
4 marketing Mrinal_Tiwari_Bhubaneshwar_1.00_yrs.pdf
5 marketing Brijesh_Pradhan_Chennai_13.00_yrs.doc
6 marketing Parvez_Ali_Hyderabad___Secunderabad_1.10_yrs.pdf
7 marketing Ravi_Anant_Sharma_Bengaluru___Bangalore_4.09_yrs.pdf
8 marketing shabaiz_khan_Bengaluru___Bangalore_4.08_yrs.doc
9 marketing Bal_Mukund_Thakur_Delhi_9.00_yrs.doc
10 marketing Sujith N V.doc
11 marketing Chandan_Das_Hyderabad___Secunderabad_10.01_yrs.doc
12 marketing Johny__Mumbai_10.00_yrs.docx
13 marketing Chitrakala_Natarajan_Chennai_7.06_yrs.docx
14 marketing Pritam_Barik_Gurgaon_3.06_yrs.pdf
15 marketing Shaleen__Bengaluru___Bangalore_10.01_yrs.docx
16 marketing shashi_kant_upadhyay_Chennai_3.01_yrs.docx
17 marketing pavan_praneeth_nallamilli_Bengaluru___Bangalore_2.04_yrs.docx
18 marketing Irfan_Shaik_Hyder

174 hr TeoPaul[4_5].docx
175 hr SimonDavisS[8_0].doc
176 hr ShelinKGeorge[6_0].pdf
177 hr GeorginaKurian[11_0].doc
178 hr MaheshMenon[15_0].doc
179 hr SureshKumarM[7_0].docx
180 hr SusindharanGB[10_0].docx
181 hr Sudhaya[11_0].pdf
182 hr NayanaDevadas[3_9].pdf
183 hr PARUCHURIRAMU[6_0].docx
184 hr SharonAnbudaiyan[1_1].doc
185 hr ShijuBhaskaran[14_0].doc
186 hr SFelixJayabalan[11_0].doc
187 hr PratapChandranK[9_0].doc
188 hr Neenu_Saseendran_Cochin___Kochi___Ernakulam_2.02_yrs.docx
189 hr PavithraB[5_5].pdf
190 hr MeenuSunil[1_1].docx
191 hr PragilP[3_0].docx
192 hr Jijokuruvilla[7_0].pdf
193 hr PradeepK[4_0].docx
194 hr KabilanSP[7_0].doc
195 hr SajeeshPV[6_0].doc
196 hr JohnsonS[17_0].docx
197 hr SwarnalathaV[4_0].docx
198 hr Nisha_Shankar.pdf
199 hr PrasadDS[13_0].docx
200 hr SrinivasKuchimanchi[15_0].docx
201 hr Swapna[10_0].doc
202 hr JayachandranR[19_0].docx
203 hr SHYAMKRISHNANS[3_0].docx
204 hr MANOJP[12_0].pdf
205 hr JERRYJPAUL[4_5].pdf
206 hr NeerajaN[2_0].doc
207 hr SalvinSi

421 seo Jayshree_Sasane_upadhyay_Pune_7.00_yrs.doc
422 seo vishnu_kumar_Delhi_2.07_yrs.docx
423 seo Divya[2_0].doc
424 seo BhuvaneswariA[1_1].docx
425 seo LakshmiRDevi[5_3].pdf
426 seo SSasikiran[2_6].doc
427 seo RajashriGHegde[1_10].doc
428 seo AbhishekC[2_3].doc
429 seo PShriVanadhi[1_0].doc
430 seo Pawan_Gupta_Gurgaon_3.11_yrs.doc
431 seo ashok__Faridabad_1.10_yrs.docx
432 seo Anil_Pratap_Singh_Delhi_4.00_yrs.docx
433 seo ShivaKumarG[4_11].docx
434 seo Avik_Bhattacharya_Bengaluru___Bangalore_2.07_yrs.pdf
435 seo SushantYM[2_7].docx
436 seo VikasRai[5_0].doc
437 seo PrincyJain[2_9].docx
438 seo Suvrangsu_Das_Kolkata_0.06_yrs.docx
439 seo KamalakarY[2_1].docx
440 seo TintuFrancis[7_0].doc
441 seo Tarun_Bhardwaj_Delhi_4.02_yrs.doc
442 seo Tanmoy_Basak_Kolkata_7.00_yrs.doc
443 seo Nikhil_Chandra_Reddy_Hyderabad___Secunderabad_3.00_yrs.doc
444 seo LakshmiVenugopal[4_0].doc
445 seo Balaji_D_Hyderabad___Secunderabad_8.02_yrs.docx
446 seo Nirup_S_Ranya_Bengaluru___Bangalore_3.02_yrs.docx
44

In [5]:
len(label_nontech)

600

In [6]:
print(len(label_nontech))


600


In [7]:
data =[]
data_labels=[]
for i,row in enumerate(label_nontech):
    data.append(row[0])
    data_labels.append(row[1])
print(len(data))
print(len(data_labels))

600
600


In [8]:
print(len(data_labels))

600


In [9]:
#labels = [0 for _ in range(len(label_resume))] + [1 for _ in range(len(label_jd))]
#print(labels)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9)

In [11]:
# Apply model on jdres train data
train_vectors = vectorizer.fit_transform(data)
print(train_vectors)

  (0, 5185)	0.0249802912163
  (0, 1476)	0.01360089906
  (0, 2057)	0.00825598965178
  (0, 5079)	0.0269265928199
  (0, 3328)	0.0377352406092
  (0, 710)	0.021605282656
  (0, 4960)	0.318937615683
  (0, 3978)	0.0966932095688
  (0, 3520)	0.0881498439991
  (0, 1039)	0.186571934994
  (0, 1820)	0.102614564246
  (0, 3213)	0.109619531579
  (0, 356)	0.100969408971
  (0, 3480)	0.11019949439
  (0, 5032)	0.0331995334789
  (0, 4440)	0.0168047173485
  (0, 45)	0.0371991379246
  (0, 6178)	0.0296647348885
  (0, 6114)	0.0563323405481
  (0, 5215)	0.0224368443958
  (0, 3647)	0.0323865064813
  (0, 5197)	0.0664405604661
  (0, 5232)	0.033086004621
  (0, 5105)	0.103970111808
  (0, 5242)	0.0177865419647
  :	:
  (599, 3006)	0.174076197584
  (599, 5536)	0.0609655008977
  (599, 4953)	0.070577895395
  (599, 33)	0.0664758297179
  (599, 804)	0.0664758297179
  (599, 1443)	0.0683551414233
  (599, 4761)	0.0664758297179
  (599, 1257)	0.070577895395
  (599, 250)	0.0664758297179
  (599, 2106)	0.0621274648718
  (599, 2565)	0.

In [12]:

X_train, X_test, y_train, y_test = train_test_split(train_vectors, data_labels, test_size=0.2, random_state=42)    

In [13]:
svm_model = svm.SVC(kernel='linear') 
svm_model.fit(X_train, y_train) 
svm_prediction = svm_model.predict(X_train)
import numpy as np
print ('svm:', np.mean(svm_prediction == y_train))

'''for i in range(80):
    print ('>>>>>>>>>>>.',model.predict(train_vectors[i]))'''

    
from sklearn.ensemble import RandomForestClassifier  
RF_model = RandomForestClassifier(max_depth=2, random_state=0)
RF_model.fit(X_train, y_train) 
RF_prediction = RF_model.predict(X_train)
import numpy as np
print ('Random Forest:', np.mean(RF_prediction == y_train))

from sklearn.linear_model import SGDClassifier
SGD_model = SGDClassifier()
SGD_model.fit(X_train, y_train) 
SGD_prediction = SGD_model.predict(X_train)
import numpy as np
print ('SGD:', np.mean(SGD_prediction == y_train))

from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train,y_train)
dtree_predictions = dtree_model.predict(X_train)
print ('DT:', np.mean(dtree_predictions == y_train))


from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train,y_train)
knn_predictions = knn.predict(X_train) 
print ('knn:', np.mean(knn_predictions == y_train))



svm: 1.0
Random Forest: 0.883333333333


/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGD: 1.0
DT: 0.7375
knn: 0.98125


In [15]:
for i in range(480):
    print ('>>>>>>>>>>>.',RF_model.predict(X_train[i]))
    print (y_train[i])
print(len(y_train))  

>>>>>>>>>>>. ['seo']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
hr
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
other
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>

>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['other']
hr
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>

In [16]:
svm_model_test = svm.SVC(kernel='linear') 
svm_model_test.fit(X_test, y_test) 
svm_prediction_test = svm_model_test.predict(X_test)
print ('svm_test:', np.mean(svm_prediction_test == y_test))

svm_test: 1.0


In [17]:
from sklearn.ensemble import RandomForestClassifier  
RF_model_test = RandomForestClassifier(max_depth=2, random_state=0)
RF_model_test.fit(X_test, y_test) 
RF_prediction_test = RF_model_test.predict(X_test)
print ('RandomForest_test:', np.mean(RF_prediction_test == y_test))

RandomForest_test: 0.866666666667


In [18]:
from sklearn.linear_model import SGDClassifier
SGD_model_test = SGDClassifier()
SGD_model_test.fit(X_test, y_test) 
SGD_prediction_test = SGD_model_test.predict(X_test)
print ('SGD_test:', np.mean(SGD_prediction_test == y_test))


SGD_test: 1.0


/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [19]:
from sklearn.tree import DecisionTreeClassifier
dtree_model_test = DecisionTreeClassifier(max_depth = 2).fit(X_test, y_test)
dtree_predictions_test = dtree_model_test.predict(X_test)
print ('DT_test:', np.mean(dtree_predictions_test == y_test))

from sklearn.neighbors import KNeighborsClassifier
knn_test = KNeighborsClassifier(n_neighbors = 7).fit(X_test, y_test)
knn_predictions_test = knn_test.predict(X_test) 
print ('knn_test:', np.mean(knn_predictions_test == y_test))


DT_test: 0.733333333333
knn_test: 0.925


In [20]:
"""for i in range(20):
    print ('>>>>>>>>>>>.',RF_model.predict(jdres_test_vectors[i]))
    print (jdres_test_labels[i])"""

for i in range(600):
    print ('>>>>>>>>>>>.',svm_model.predict(train_vectors[i]))
    print (data_labels[i])


>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>

seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['seo']


In [21]:
for i in range(120):
    print('>>>>>>>>>>>.',SGD_model_test.predict(X_test[i])) 
    print (y_test[i])

>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['marketing']
marketing
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['seo']
seo
>>>>>>>>>>>. ['other']
other
>>>>>>>>>>>. ['hr']
hr
>>>>>>>>>>>. ['marketing']
marke

In [22]:
svm_model_check = svm.SVC(kernel='linear') 
svm_model_check.fit(X_train, y_train) 
svm_prediction_check = svm_model_check.predict(X_test)
print ('svm_test:', np.mean(svm_prediction_check == y_test))

svm_test: 0.933333333333


In [23]:
SGD_model_check = SGDClassifier()
SGD_model_check.fit(X_train, y_train) 
SGD_prediction_check = SGD_model_check.predict(X_test)
print ('SGD_test:', np.mean(SGD_prediction_check == y_test))


SGD_test: 0.941666666667


/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [24]:
from sklearn.ensemble import RandomForestClassifier  
RF_model_test = RandomForestClassifier(max_depth=2, random_state=0)
RF_model_test.fit(X_train, y_train) 
RF_prediction_test = RF_model_test.predict(X_test)
print ('RandomForest_test:', np.mean(RF_prediction_test == y_test))

from sklearn.tree import DecisionTreeClassifier
dtree_model_check = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)
dtree_predictions_check = dtree_model_check.predict(X_test)
print ('DT_test:', np.mean(dtree_predictions_check == y_test))

from sklearn.neighbors import KNeighborsClassifier
knn_check = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train)
knn_predictions_check = knn_check.predict(X_test) 
print ('knn_test:', np.mean(knn_predictions_check == y_test))

RandomForest_test: 0.858333333333
DT_test: 0.7
knn_test: 0.941666666667


In [25]:
target_names=['Resume', 'JobDescription']

In [26]:

y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       1.00      0.93      0.96        29
JobDescription       0.88      0.91      0.90        33

   avg / total       0.94      0.93      0.93       120



/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 4, does not match size of target_names, 2
  .format(len(labels), len(target_names))


In [27]:

y_pred = RF_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       0.96      0.83      0.89        29
JobDescription       0.86      0.76      0.81        33

   avg / total       0.87      0.86      0.86       120



/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 4, does not match size of target_names, 2
  .format(len(labels), len(target_names))


In [28]:

y_pred = SGD_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       0.97      0.97      0.97        29
JobDescription       0.88      0.91      0.90        33

   avg / total       0.93      0.93      0.93       120



/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 4, does not match size of target_names, 2
  .format(len(labels), len(target_names))


In [29]:

y_pred = dtree_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       0.96      0.90      0.93        29
JobDescription       1.00      0.03      0.06        33

   avg / total       0.85      0.70      0.63       120



/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 4, does not match size of target_names, 2
  .format(len(labels), len(target_names))


In [30]:
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       0.97      1.00      0.98        29
JobDescription       0.86      0.94      0.90        33

   avg / total       0.95      0.94      0.94       120



/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 4, does not match size of target_names, 2
  .format(len(labels), len(target_names))


In [31]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])
SGD_model_test.predict(test_file)[0]

'other'

In [32]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])
svm_model.predict(test_file)[0]

'other'

In [33]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])

RF_model.predict(test_file)[0]

'other'

In [34]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])
dtree_model.predict(test_file)[0]

'other'

In [35]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])
knn.predict(test_file)[0]

'other'